In [2]:
from givernylocal.turbulence_dataset import *
from givernylocal.turbulence_toolkit import *
import json
import os
from tqdm import tqdm
import contextlib
from dotenv import load_dotenv

load_dotenv()

auth_token = os.getenv('auth_token')
dataset_title = 'isotropic1024coarse'
output_path = './giverny_output'

# instantiate the dataset.
dataset = turb_dataset(dataset_title = dataset_title, output_path = output_path, auth_token = auth_token)

In [3]:
@contextlib.contextmanager
def suppress_stdout():
    with open(os.devnull, 'w') as devnull:
        with contextlib.redirect_stdout(devnull):
            yield

In [4]:
variables = ['velocity', 'pressure', 'force']
temporal_method = 'none'
spatial_method = 'lag8'
spatial_operator = 'field'

In [23]:
temporal_method_tmp = 'pchip'
time_start = 0.1
time_end = 0.9
delta_t = 0.008
nsteps = int((time_end - time_start)/delta_t) + 1
times = [time_start + i*delta_t for i in range(nsteps)]
# option = [time_end, delta_t]

nx = 64
ny = 64
nz = 64
n_points = nx * ny * nz

x_points = np.linspace(1.0, 1.6, nx, dtype = np.float64)
y_points = np.linspace(-0.9, -0.3, ny, dtype = np.float64)
z_points = np.linspace(0.5, 1.1, nz, dtype = np.float64)

points = np.array([axis.ravel() for axis in np.meshgrid(x_points, y_points, z_points, indexing = 'ij')], dtype = np.float64).T

output_dir = "turbulence_timesteps"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
positions = getData(dataset, 'position', times[0], temporal_method_tmp, spatial_method, spatial_operator, points, [time_end, delta_t])
with open(os.path.join(output_dir, 'positions.json'), 'w') as f:
    json.dump(np.array(positions).tolist(), f)

In [ ]:
for time in tqdm(times):
    data = {} 
    for variable in variables:
        with suppress_stdout():
            if variable == 'force':
                data[variable] = getData(dataset, variable, time, temporal_method_tmp, 'none', spatial_operator, points)
            else:
                data[variable] = getData(dataset, variable, time, temporal_method_tmp, spatial_method, spatial_operator, points)

    data_output = {key: np.array(value).tolist() for key, value in data.items()}
    filename = os.path.join(output_dir, f"timestep_{time:.3f}.json")
    with open(filename, 'w') as f:
        json.dump(data_output, f)

  1%|          | 1/101 [02:46<4:37:39, 166.60s/it]